In [1]:
%cd ..

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/dataset_curation


In [2]:
import pandas as pd
import scripts

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/venv/lib/python3.12/site-packages/goatools/__init__.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/dataset_curation/scripts/../dependencies/go-basic.obo: fmt(1.2) rel(2025-07-22) 43,230 Terms


In [3]:
# ENV
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
df = pd.read_csv(getenv("FETCHED_DATA"), index_col="Entry")
df.drop(columns=["Protein families"], inplace=True)

In [5]:
scripts.clean_all_cols(df, inplace=True)
"The output was suppressed"

No preprocessing rule for 'Sequence' — leaving as is.


'The output was suppressed'

In [6]:
df = df.loc[df.notna().all(axis=1)].copy()

# Feature Engineering

In [7]:
# Inits
txt_embedder = scripts.FreeTXTEmbedder(getenv("OPENAI_API_KEY"), "SMALL_OPENAI_MODEL",
                                       cache_file_path="testing_cache", caching_mode="CREATE/OVERRIDE")
aa_embedder = scripts.AAChainEmbedder("ESM2")

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Converting columns to vectors

In [8]:
scripts.embed_ft_domains(df, aa_embedder, inplace=True)
scripts.embed_freetxt_cols(df, ["Domain [CC]", "Function [CC]", "Catalytic activity", "Pathway"], txt_embedder, inplace=True)
scripts.encode_go(df, "Gene Ontology (molecular function)", coverage_target=0.9, inplace=True)
scripts.encode_go(df, "Gene Ontology (biological process)", coverage_target=0.9, inplace=True)
scripts.encode_ec(df, coverage_target=0.9, inplace=True)
scripts.encode_multihot(df, "Rhea ID", inplace=True)
scripts.encode_multihot(df, "Cofactor", inplace=True)
"The output was suppressed"

'The output was suppressed'

In [9]:
df

,Domain [FT],Domain [CC],Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Rhea ID,Cofactor
Entry,,,,,,,,,,
A0A174S1W4,"[0.06519439, -0.59267235, 0.39969912, 0.265044...","[0.015534245, 0.012767873, 0.03198688, 0.09134...","[1, 0, 0, 0]","[0, 0, 1, 0, 0]","[0.03972639, 0.023883, 0.020913705, 0.05930016...","[0.0010682484, -0.013508173, 0.018412925, 0.00...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-0.039731007, -0.01863717, -0.005424209, 0.00...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 1, 0]"
A1A0T4,"[0.07417185, -0.603595, 0.40701595, 0.2565485,...","[0.03333539, 0.038547587, 0.0024857074, 0.0261...","[1, 1, 0, 0]","[1, 1, 1, 0, 0]","[0.011446279, 0.029901857, 0.02459995, 0.02289...","[0.019830298, -0.0031177108, 0.0076907477, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]","[0.0015797128, -0.01947734, -0.03513559, -0.00...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0]"
A0A0M6WK58,"[0.07809408, -0.60110337, 0.40830672, 0.252053...","[0.0073871133, 0.012897757, 0.06351492, 0.0227...","[1, 1, 0, 0]","[0, 0, 1, 0, 0]","[-0.010897795, 0.014558259, 0.035493966, 0.016...","[0.019652143, -0.020094935, 0.004633035, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.004218073, 0.0016776051, 0.0033187044, 0.03...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]"
R9IM47,"[0.07626842, -0.60418457, 0.40725395, 0.255790...","[0.03333539, 0.038547587, 0.0024857074, 0.0261...","[1, 1, 0, 0]","[1, 1, 1, 0, 0]","[0.011446279, 0.029901857, 0.02459995, 0.02289...","[0.019830298, -0.0031177108, 0.0076907477, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1]","[0.0015797128, -0.01947734, -0.03513559, -0.00...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0]"
F3AQF4,"[0.08569833, -0.600624, 0.42240095, 0.24795365...","[-0.011031064, -0.030502003, 0.042825814, 0.00...","[1, 0, 1, 0]","[0, 0, 1, 0, 0]","[0.035522394, 0.033507623, -0.017557321, 0.021...","[0.02502062, 0.023106474, -0.018286929, 0.0654...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0.0058564125, 0.012284613, -0.026054699, 0.03...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]","[1, 0, 0, 0, 0, 0, 0]"
R8W4J3,"[0.07836224, -0.6018773, 0.40831724, 0.2527847...","[0.0073871133, 0.012897757, 0.06351492, 0.0227...","[1, 1, 0, 0]","[0, 0, 1, 0, 0]","[-0.010897795, 0.014558259, 0.035493966, 0.016...","[0.019652143, -0.020094935, 0.004633035, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.004218073, 0.0016776051, 0.0033187044, 0.03...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]"
A0A174LTZ8,"[0.07836224, -0.6018773, 0.40831724, 0.2527847...","[0.0073871133, 0.012897757, 0.06351492, 0.0227...","[1, 1, 0, 0]","[0, 0, 1, 0, 0]","[-0.010897795, 0.014558259, 0.035493966, 0.016...","[0.019652143, -0.020094935, 0.004633035, 0.024...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[0.004218073, 0.0016776051, 0.0033187044, 0.03...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0]"
Q54089,"[0.09226795, -0.58187866, 0.43748796, 0.23752,...","[0.02599805, -0.0066759973, 0.0365628, 0.01812...","[1, 1, 1, 0]","[0, 0, 0, 0, 1]","[0.018235786, 0.02937339, 0.041911952, 0.01577...","[0.0038128197, 0.017893419, 0.027600924, 0.019...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[-0.0299906, 0.028268652, 0.032262616, 0.01207...","[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0]"
P11454,"[0.10389228, -0.5899302, 0.4392348, 0.23114714...","[0.035087395, 0.0027331295, -0.017778553, 0.01...","[1, 1, 0, 0]","[0, 0, 1, 0, 0]","[0.032303397, -0.040006146, -0.0073055746, 0.0...","[0.01471096, -0.06492091, -0.009152663, 0.0091...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0]","[0.010810949, -0.042123366, -0.01407688, 0.006...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 1]"
